In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [53]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches_1.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

# Calculate Team Strength

In [54]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Afghanistan,0.916667,1.833333
Albania,1.205882,1.294118
Algeria,1.780488,0.951220
Andorra,0.526316,1.868421
Angola,0.966667,0.933333
...,...,...
Yemen,0.533333,2.400000
Yugoslavia,1.666667,1.272727
Zaire,0.000000,4.666667


# Function predict_points

In [55]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

## Testing function

In [56]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(1.821416031525293, 0.9695638673936005)
(1.902196894343332, 0.8924309016942673)
(0, 0)


# Predicting World Cup

## Group stage

In [57]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [58]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [59]:
dict_table['Group H']

,Team,Pts
0,Portugal,6.0
1,Ghana,4.0
2,Uruguay,4.0
3,South Korea,3.0


In [60]:
teams_in_group

array(['Portugal', 'Ghana', 'Uruguay', 'South Korea'], dtype=object)

## Knock out

In [61]:
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [62]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Iran,2022,?
49,Argentina,Match 50,France,2022,?
50,Denmark,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Morocco,2022,?
53,Brazil,Match 54,Ghana,2022,?
54,Canada,Match 55,Spain,2022,?
55,Portugal,Match 56,Serbia,2022,?


In [63]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [64]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Iran,2022,Netherlands
49,Argentina,Match 50,France,2022,France
50,Denmark,Match 52,Poland,2022,Denmark
51,England,Match 51,Senegal,2022,Senegal
52,Germany,Match 53,Morocco,2022,Morocco
53,Brazil,Match 54,Ghana,2022,Brazil
54,Canada,Match 55,Spain,2022,Canada
55,Portugal,Match 56,Serbia,2022,Portugal


## Quarter Final

In [65]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [66]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Morocco,Match 58,Brazil,2022,?
57,Netherlands,Match 57,France,2022,?
58,Canada,Match 60,Portugal,2022,?
59,Senegal,Match 59,Denmark,2022,?


In [67]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Morocco,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,France,2022,Netherlands
58,Canada,Match 60,Portugal,2022,Canada
59,Senegal,Match 59,Denmark,2022,Senegal


## Semifinal

In [68]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,Senegal,Match 62,Canada,2022,?


In [69]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,Senegal,Match 62,Canada,2022,Canada


## Final

In [70]:
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,Canada,2022,?


In [71]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,Canada,2022,Canada
